In [1]:
### Required libraries ###

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Dropout, Dense, Input, BatchNormalization, Activation, Add, LSTM, Softmax, Bidirectional, Conv1D
from keras.optimizers import Adam
from keras.utils import to_categorical, normalize
from keras import backend as K

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
### Dataset ###

# reading dataset
dataset = pd.read_csv(r"D:\Data\Dinesh\Work\DLTraining\UCI Credit Card - Demographics Model\dataset\UCI_Credit_Card.csv", index_col=0)
dataset["AGE_CAT"] = pd.cut(dataset["AGE"], [0, 15, 25, 35, 45, 55, 80], labels=[15, 25, 35, 45, 55, 80])

neg_num_categorical_columns = ["PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
for column in neg_num_categorical_columns:
    min_value = abs(dataset[column].min())
    dataset[column] += abs(min_value)

In [5]:
### Data Preprocessing ###

dataset = np.array(dataset)

X_payment = normalize(np.array(dataset[:, 17:22], dtype="float32").reshape(30000, 5, 1))
Y_payment = normalize(np.array(dataset[:, 22], dtype="float32").reshape(30000, 1))

X_default = normalize(np.array(dataset[:, 17:23], dtype="float32").reshape(30000 , 6, 1))
Y_default = to_categorical(np.array(dataset[:, 23], dtype="float32")).reshape(30000, 2)

In [7]:
### Train and Test set preparation ###

X_payment_temp, X_payment_test, Y_payment_temp, Y_payment_test = train_test_split(X_payment, Y_payment, test_size=0.10, random_state=5)
X_payment_train, X_payment_dev, Y_payment_train, Y_payment_dev = train_test_split(X_payment_temp, Y_payment_temp, test_size=0.10, random_state=5)
X_default_temp, X_default_test, Y_default_temp, Y_default_test = train_test_split(X_default, Y_default, test_size=0.10, random_state=5)
X_default_train, X_default_dev, Y_default_train, Y_default_dev = train_test_split(X_default_temp, Y_default_temp, test_size=0.10, random_state=5)

In [23]:
### Model ###

def build_model(layers_dims, n_X, n_Y, output_activation, loss_function):
    
    # input layer
    X_input = Input(shape = (n_X,))
    A = X_input

    # hidden layer
    # list having no. of activations in each layer
    add_this_a = None
    fix_add_this_a = False
#     layers_dims = [8, 12, 16, 16, 12, 8]
    for i, dim in enumerate(layers_dims):

        layer_no = i + 1

#         skip_add = True if (layer_no%3 == 0 and layer_no/3 > 1) else False # for skip connections, every 3 layers
#         add_this_A = True if (layer_no + 2)%3 == 0 else False # for skip connections

        A = Dense(dim, activation = "linear")(A) # to change activation check here, https://keras.io/activations/

#         if skip_add == True: # for skip connections
#             Add()([A, to_add_A])

        A = Activation("relu")(A)

#         if add_this_A == True: # for skip connections
#             to_add_A = A

        A = BatchNormalization()(A)
    #     A = Dropout(0.7)(A) # add dropout, if needed


    Y_layer = Dense(n_Y, activation=output_activation)(A)
    # Y_layer = Dense(1, activation="linear")(A)

    model = Model(inputs = [X_input], outputs = [Y_layer])

    # adam optimizer - try different learning rates here
    opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.002)
    model.compile(loss=loss_function, optimizer=opt, metrics=['accuracy'])
    # model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
    
    return model

# prints model graph
# model.summary()

In [24]:
# X and Y for the current run, and their dimensions

X_train = X_payment_train
Y_train = Y_payment_train
X_test = X_payment_test
Y_test = Y_payment_test
m = X_train.shape[0]
n_X = X_train.shape[1]
n_X_dim = X_train.shape[2]
n_Y = Y_train.shape[1]

# clearing session
K.clear_session()

# build model with a set of dimensions
model = build_model([], n_X, n_Y, "linear", "mean_squared_error")
# print out the model layers
# model.summary()

# training model
model.fit(X_train.reshape(m, n_X), Y_train, epochs=5, batch_size=64)
model.evaluate(X_test.reshape(n_X, -1).T, Y_test)

Epoch 1/5
24300/24300 [==============================] - 0s 12us/step - loss: 0.1977 - acc: 0.7714
Epoch 2/5
24300/24300 [==============================] - 0s 8us/step - loss: 0.1208 - acc: 0.8415
Epoch 3/5
24300/24300 [==============================] - 0s 10us/step - loss: 0.1201 - acc: 0.8435

[0.2067277268966039, 0.737666666507721]

In [25]:
# X and Y for the current run, and their dimensions

X_train = X_default_train
Y_train = Y_default_train
X_test = X_default_test
Y_test = Y_default_test
m = X_train.shape[0]
n_X = X_train.shape[1]
n_X_dim = X_train.shape[2]
n_Y = Y_train.shape[1]

# clearing session
K.clear_session()

# build model with a set of dimensions
model = build_model([], n_X, n_Y, "sigmoid", "categorical_crossentropy")
# print out the model layers
# model.summary()

# training model
model.fit(X_train.reshape(m, n_X), Y_train, epochs=5, batch_size=64)
model.evaluate(X_test.reshape(n_X, -1).T, Y_test)

Epoch 1/5
24300/24300 [==============================] - 0s 13us/step - loss: 0.5173 - acc: 0.7781
Epoch 2/5
24300/24300 [==============================] - 0s 8us/step - loss: 0.5133 - acc: 0.7788
Epoch 3/5
24300/24300 [==============================] - 0s 10us/step - loss: 0.5123 - acc: 0.7788

[0.5398216325441997, 0.7806666668256124]